In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!pip install emot
!pip install emoji
!pip install demoji
!pip install punct
!pip install contractions


ERROR: Could not find a version that satisfies the requirement punct (from versions: none)
ERROR: No matching distribution found for punct


In [ ]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import emot
import nltk.data
import numpy as np
import re
import time
import string
import emoji
import demoji
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import contractions
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
from nltk.tokenize import TweetTokenizer
import regex
import nltk
from nltk.stem import SnowballStemmer
demoji.download_codes()
lemmatizer = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,accuracy_score
from sklearn.svm import LinearSVC


<ipython-input-33-f1fe1e21334d>:25: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
data=pd.read_csv("/content/drive/MyDrive/NLP project/tulu-train_set.csv")
dev=pd.read_csv("/content/drive/MyDrive/NLP project/Tulu_dev.csv")
data

,Text,Annotations
0,Devdas kapikad no1,Positive
1,ajji eer super,Positive
2,Bega 2 nd part padle,Neutral
3,ynkul Kudla dakul yepala Ideal Icecream g 10 o...,Neutral
4,Tulu panda enku masth ista i love tulu tulunadu,Positive
...,...,...
6940,I will support you always Rakshita 💕 #masthLik...,Positive
6941,Waltranna bari edde programme.,Positive
6942,"Solmelu 🙏 Mangaloreg ""Kudla""panre... Thank you...",Neutral
6943,Super bit super,Positive


**Summary of the dataset**
for #**TULU**

In [ ]:
#Summary of the dataset
data.describe()

,Text,Annotations
count,6945,6945
unique,6773,4
top,Maatherla tulu chithrarangan pugarnaga masth p...,Positive
freq,3,3352


In [ ]:
#sentiment count
data['Annotations'].value_counts()

Positive         3352
Neutral          1854
Mixed Feeling    1041
Negative          698
Name: Annotations, dtype: int64

In [ ]:
def emo(text):
    emoji_list = [c for c in text if emoji.is_emoji(c)]
    for emoji_char in emoji_list:
        text = text.replace(emoji_char, emoji_char +" ")
    return text

In [ ]:
def preprocess(text):
  text=emo(text)
  text = contractions.fix(text.lower(), slang=True)
  text=text.lower()
  exclude = set(string.punctuation)
  text =re.sub("@ ?[A-Za-z0-9_]+", "", text)
  text= re.sub(r'\d+', '', text)
  text=re.sub(r's*$','', text)
  text=emoji.demojize(text)
  text=re.sub(r'^rt\s+','',text)
  text=re.sub(r'@\w+','',text)
  text=re.sub(r'https\S+','',text)
  text=re.sub(r'‘','',text)
  text=re.sub(r'\s+',' ',text)

  text = ''.join(ch for ch in text if ch not in exclude)

  return text


In [ ]:
data['clean_text']=data["Text"].apply(lambda x:emo(x))
data["clean_text"]=data['clean_text'].apply(lambda X: preprocess(X))
data

,Text,Annotations,clean_text
0,Devdas kapikad no1,Positive,devdas kapikad no
1,ajji eer super,Positive,ajji ever super
2,Bega 2 nd part padle,Neutral,bega nd part padle
3,ynkul Kudla dakul yepala Ideal Icecream g 10 o...,Neutral,ynkul kudla dakul yepala ideal icecream g out ...
4,Tulu panda enku masth ista i love tulu tulunadu,Positive,tulu panda enku masth ista i love tulu tulunadu
...,...,...,...
6940,I will support you always Rakshita 💕 #masthLik...,Positive,i will support you always rakshita twohearts m...
6941,Waltranna bari edde programme.,Positive,waltranna bari edde programme
6942,"Solmelu 🙏 Mangaloreg ""Kudla""panre... Thank you...",Neutral,solmelu foldedhands mangaloreg kudlapanre than...
6943,Super bit super,Positive,super bit super


In [ ]:
dev['clean_text']=dev["Text"].apply(lambda x:emo(x))
dev["clean_text"]=dev['clean_text'].apply(lambda X: preprocess(X))

FEATURE EXTRACTION

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
TfidfVectorizer?

In [ ]:
tf_idf = TfidfVectorizer()
#applying tf idf to training data
X_train_tf = tf_idf.fit_transform(data['clean_text'])
#applying tf idf to training data
X_train_tf = tf_idf.transform(data['clean_text'])

In [ ]:
X_train_tf

<6945x12445 sparse matrix of type '<class 'numpy.float64'>'
	with 42077 stored elements in Compressed Sparse Row format>

In [ ]:
#transforming test data into tf-idf matrix
X_test_tf = tf_idf.transform(dev["clean_text"])

In [ ]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, data['Annotations'])

MultinomialNB()

In [ ]:
y_pred = naive_bayes_classifier.predict(X_test_tf)
y_pred

array(['Neutral', 'Positive', 'Positive', 'Neutral', 'Positive',
       'Positive', 'Positive', 'Neutral', 'Positive', 'Positive',
       'Neutral', 'Positive', 'Positive', 'Positive', 'Neutral',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Neutral', 'Positive', 'Neutral', 'Neutral',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Neutral', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Neutral', 'Positive', 'Positive', 'Positive',
       'Neutral', 'Neutral', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Neutral',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Positive', 'Positive', 'Positive', 'Negative', 'Positive',
       'Neutral', 'Positive', 'Positive', 'Positive', 'Neutral',
       'Positive', 'Positive', 'Neutral', 'Neutral', 'Positive',
       'Positive', 'Positive', 'Positive', 'Positive', 'Positive',
       'Posi

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
dev['Annotations']

0           Positive
1      Mixed Feeling
2           Negative
3            Neutral
4           Positive
           ...      
495    Mixed Feeling
496          Neutral
497         Positive
498          Neutral
499         Positive
Name: Annotations, Length: 500, dtype: object

In [ ]:
print(classification_report(dev['Annotations'], y_pred))

               precision    recall  f1-score   support

Mixed Feeling       0.50      0.01      0.02        90
     Negative       0.83      0.09      0.16        55
      Neutral       0.53      0.37      0.44       124
     Positive       0.56      0.99      0.72       231

     accuracy                           0.56       500
    macro avg       0.61      0.37      0.33       500
 weighted avg       0.57      0.56      0.46       500



In [ ]:
from sklearn.svm import LinearSVC

lsvc=LinearSVC()
lsvc.fit(X_train_tf, data['Annotations'])
lsvc_pred=lsvc.predict(X_test_tf)
print(classification_report(dev['Annotations'], lsvc_pred))

               precision    recall  f1-score   support

Mixed Feeling       0.33      0.20      0.25        90
     Negative       0.50      0.36      0.42        55
      Neutral       0.51      0.62      0.56       124
     Positive       0.75      0.83      0.79       231

     accuracy                           0.61       500
    macro avg       0.52      0.50      0.51       500
 weighted avg       0.59      0.61      0.60       500



In [ ]:
from sklearn import svm
svm=svm.SVC()
svm.fit(X_train_tf, data['Annotations'])
svm_pred=svm.predict(X_test_tf)
print(classification_report(dev['Annotations'], svm_pred))

               precision    recall  f1-score   support

Mixed Feeling       0.44      0.08      0.13        90
     Negative       0.76      0.24      0.36        55
      Neutral       0.52      0.71      0.60       124
     Positive       0.70      0.91      0.79       231

     accuracy                           0.64       500
    macro avg       0.61      0.48      0.47       500
 weighted avg       0.62      0.64      0.58       500



In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train_tf, data['Annotations'])
clf_pred=clf.predict(X_test_tf)
print(classification_report(dev['Annotations'], clf_pred))


               precision    recall  f1-score   support

Mixed Feeling       0.36      0.23      0.28        90
     Negative       0.34      0.20      0.25        55
      Neutral       0.47      0.44      0.45       124
     Positive       0.67      0.84      0.75       231

     accuracy                           0.56       500
    macro avg       0.46      0.43      0.43       500
 weighted avg       0.53      0.56      0.54       500



In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
clf.fit(X_train_tf, data['Annotations'])
clf_pred=clf.predict(X_test_tf)
print(classification_report(dev['Annotations'], clf_pred))


               precision    recall  f1-score   support

Mixed Feeling       0.34      0.11      0.17        90
     Negative       0.68      0.31      0.42        55
      Neutral       0.54      0.69      0.61       124
     Positive       0.72      0.89      0.79       231

     accuracy                           0.64       500
    macro avg       0.57      0.50      0.50       500
 weighted avg       0.60      0.64      0.59       500



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train_tf, data['Annotations'])
neigh_pred=clf.predict(X_test_tf)
print(classification_report(dev['Annotations'], neigh_pred))

               precision    recall  f1-score   support

Mixed Feeling       0.34      0.11      0.17        90
     Negative       0.68      0.31      0.42        55
      Neutral       0.54      0.69      0.61       124
     Positive       0.72      0.89      0.79       231

     accuracy                           0.64       500
    macro avg       0.57      0.50      0.50       500
 weighted avg       0.60      0.64      0.59       500



In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
voting = VotingClassifier(
    estimators=[
        ('linear_svc', lsvc),
        ('multinomial', naive_bayes_classifier),
        ('support_vector_machine', svm)
    ],
    voting='hard'
)

In [ ]:
voting.fit(X_train_tf, data['Annotations'])
y_pred_voting = voting.predict(X_test_tf)
print("Voting Classifier Report:")
print(classification_report(dev['Annotations'], y_pred_voting))


Voting Classifier Report:
               precision    recall  f1-score   support

Mixed Feeling       0.38      0.13      0.20        90
     Negative       0.61      0.31      0.41        55
      Neutral       0.54      0.56      0.55       124
     Positive       0.68      0.92      0.78       231

     accuracy                           0.62       500
    macro avg       0.55      0.48      0.49       500
 weighted avg       0.58      0.62      0.58       500



In [ ]:
classifiers = [lsvc, naive_bayes_classifier, svm]
for clf in classifiers:
    clf.fit(X_train_tf, data['Annotations'])
    y_pred_clf = clf.predict(X_test_tf)
    clf_name = clf.__class__.__name__
    print(f"{clf_name} Report:")
    print(classification_report(dev['Annotations'], y_pred_clf))

LinearSVC Report:
               precision    recall  f1-score   support

Mixed Feeling       0.33      0.20      0.25        90
     Negative       0.50      0.36      0.42        55
      Neutral       0.51      0.62      0.56       124
     Positive       0.75      0.83      0.79       231

     accuracy                           0.61       500
    macro avg       0.52      0.50      0.51       500
 weighted avg       0.59      0.61      0.60       500

MultinomialNB Report:
               precision    recall  f1-score   support

Mixed Feeling       0.50      0.01      0.02        90
     Negative       0.83      0.09      0.16        55
      Neutral       0.53      0.37      0.44       124
     Positive       0.56      0.99      0.72       231

     accuracy                           0.56       500
    macro avg       0.61      0.37      0.33       500
 weighted avg       0.57      0.56      0.46       500

SVC Report:
               precision    recall  f1-score   support

Mixed 

In [ ]:
model = ClassificationModel('bert', 'distilbert-base-multilingual-cased', num_labels=4, args={'reprocess_input_data': True, 'overwrite_output_dir': True}, ignore_mismatched_sizes=True)